In [21]:
import pandas as pd
import re
from datetime import datetime

## 내부 데이터

In [22]:
# 데이터 로드

folder = '235724_한국 부동산 데이터 시각화 경진대회_data/'

data_permit = pd.read_csv(folder + '건축허가현황.csv', index_col=0)
data_ordamt = pd.read_csv(folder + '국내건설수주액.csv', index_col=0)
data_unsold = pd.read_csv(folder + '미분양주택현황.csv', index_col=0)
data_rprice = pd.read_csv(folder + '아파트 실거래가격지수.csv', index_col=0)
data_mprice = pd.read_csv(folder + '유형별_주택월세통합가격지수.csv', index_col=0)
data_permit2 = pd.read_csv(folder + '주택건설인허가실적.csv', index_col=0)
data_kb1 = pd.read_csv(folder + '주택매매가격지수(KB).csv', index_col=0)
data_kb2 = pd.read_csv(folder + '주택전세가격지수(KB).csv', index_col=0)
data_cprice = pd.read_csv(folder + '지역별_지가변동률.csv', index_col=0)

datas = [data_permit, data_ordamt, data_unsold, data_rprice, data_mprice, data_permit2,
             data_kb1, data_kb2, data_cprice]
data_names = ['건축허가현황', '국내건설수주액', '미분양주택현황', '아파트 실거래가격지수', '유형별_주택월세통합가격지수',
              '주택건설인허가실적', '주택매매가격지수(KB)', '주택전세가격지수(KB)', '지역별_지가변동률']

In [23]:
# 데이터 살펴보기

'''
for i in range(len(datas)):
    print(f'*****{data_names[i]}*****')
    display(datas[i].head())
    print(datas[i].info())
    print(datas[i].index)
    print('\n\n\n')
'''

"\nfor i in range(len(datas)):\n    print(f'*****{data_names[i]}*****')\n    display(datas[i].head())\n    print(datas[i].info())\n    print(datas[i].index)\n    print('\n\n\n')\n"

In [24]:
# 통합 파일 1 만들기 
# 지역별 주택 지표 변화 흐름을 보여주는 대시보드 구성
# 데이터 전처리

# 지역 컬럼명 정제
data_unsold.columns = [re.sub('\[.+\]', '', x) for x in data_unsold.columns]
data_rprice.columns = [re.sub('\[.+\]', '', x) for x in data_rprice.columns]
data_permit2.columns = [re.sub('\[.+\]', '', x) for x in data_permit2.columns]
data_cprice.columns = [re.sub('\[.+\]', '', x) for x in data_cprice.columns]

# 유형별_주택월세통합가격지수 유형 분리 후 지역 정제
CNUM = 24
data_mprice.columns = [re.sub('\[.+\]', '', x) for x in data_mprice.columns]
data_mprice_1 = data_mprice.iloc[:, :CNUM]  # 종합
data_mprice_2 = data_mprice.iloc[:, CNUM:CNUM*2]  # 아파트
data_mprice_3 = data_mprice.iloc[:, CNUM*2:CNUM*3]  # 연립다세대
data_mprice_4 = data_mprice.iloc[:, CNUM*3:CNUM*4]  # 단독주택

data_mprice_1.columns = [re.sub('.+-', '', x) for x in data_mprice_1.columns]
data_mprice_2.columns = [re.sub('.+-', '', x) for x in data_mprice_2.columns]
data_mprice_3.columns = [re.sub('.+-', '', x) for x in data_mprice_3.columns]
data_mprice_4.columns = [re.sub('.+-', '', x) for x in data_mprice_4.columns]

# 지역별_지가변동률 컬럼 기준으로 통합하기 위해 없는 컬럼 쳐내고 구분 컬럼 추가
data_unsold.drop(list(set(data_unsold.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_unsold['구분'] = '미분양주택현황'
data_rprice.drop(list(set(data_rprice.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_rprice['구분'] = '아파트실거래가격지수'
data_permit2.drop(list(set(data_permit2.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_permit2['구분'] = '주택건설인허가실적'
data_cprice['구분'] = '지역별지가변동률'
data_mprice_1.drop(list(set(data_mprice_1.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_mprice_1['구분'] = '종합월세통합가격지수'
data_mprice_2.drop(list(set(data_mprice_2.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_mprice_2['구분'] = '아파트월세통합가격지수'
data_mprice_3.drop(list(set(data_mprice_3.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_mprice_3['구분'] = '연립다세대월세통합가격지수'
data_mprice_4.drop(list(set(data_mprice_4.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_mprice_4['구분'] = '단독주택월세통합가격지수'

# 통합파일 만들기
merge_data1 = pd.concat([data_unsold, data_rprice, data_permit2, data_cprice,
                         data_mprice_1, data_mprice_2, data_mprice_3, data_mprice_4], axis=0)
merge_data1.index.name = '날짜'
merge_data1.reset_index(inplace=True)

# 데이터 구조 변경
merge_data1 = pd.melt(merge_data1, id_vars=['날짜', '구분'], var_name='지역', value_name='수치')
merge_data1.set_index('날짜', inplace=True)
merge_data1

C:\Users\donguk\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-24-fa3f9662524b>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_mprice_1['구분'] = '종합월세통합가격지수'
<ipython-input-24-fa3f9662524b>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

,구분,지역,수치
날짜,,,
2007-01-01,미분양주택현황,전국,75616.0
2007-02-01,미분양주택현황,전국,73619.0
2007-03-01,미분양주택현황,전국,73162.0
2007-04-01,미분양주택현황,전국,73393.0
2007-05-01,미분양주택현황,전국,78571.0
...,...,...,...
2020-11-01,단독주택월세통합가격지수,세종,100.2
2020-12-01,단독주택월세통합가격지수,세종,100.3
2021-01-01,단독주택월세통합가격지수,세종,100.4


In [25]:
# 통합 파일 2 만들기
# 주택매매가격지수 & 주택전세가격지수 통합

# 컬럼명 정제
data_kb1.columns = [re.sub('\[.+\]', '', x) for x in data_kb1.columns]
data_kb2.columns = [re.sub('\[.+\]', '', x) for x in data_kb2.columns]

# 구분 컬럼 추가
data_kb1['구분'] = '주택매매가격지수'
data_kb2['구분'] = '주택전세가격지수'

# 통합파일 만들기
merge_data2 = pd.concat([data_kb1, data_kb2], axis=0)
merge_data2.index.name = '날짜'
merge_data2

,총지수,단독주택,연립주택,아파트,아파트(서울),총지수(서울),구분
날짜,,,,,,,
1986-01-01,34.656,61.347,43.053,20.973,17.153,30.044,주택매매가격지수
1986-02-01,34.656,61.347,42.925,20.973,17.183,30.044,주택매매가격지수
1986-03-01,34.708,61.513,42.733,20.935,17.153,30.002,주택매매가격지수
1986-04-01,34.449,61.097,42.733,20.819,17.034,29.837,주택매매가격지수
1986-05-01,34.293,60.848,41.837,20.664,16.945,29.588,주택매매가격지수
...,...,...,...,...,...,...,...
2020-11-01,104.886,102.823,105.011,105.140,110.216,109.252,주택전세가격지수
2020-12-01,106.043,103.249,105.711,106.608,112.406,110.887,주택전세가격지수
2021-01-01,106.925,103.545,106.306,107.688,114.116,112.229,주택전세가격지수


In [26]:
# 파일 추출
# Tableau Dashboard로 시각화 진행

merge_data1.to_csv('merge_data_1.csv', encoding='cp949')
merge_data2.to_csv('merge_data_2.csv', encoding='cp949')

## 외부 데이터
1. 통계청 '행정구역 시군구별, 성별 인구수'
2. 통계청 '시군구별 이동자수'

In [27]:
# 데이터 로드

data_pop = pd.read_csv('행정구역_시군구_별__성별_인구수.csv', encoding='cp949')
data_mv = pd.read_csv('시군구별_이동자수.csv', encoding='cp949')

In [28]:
data_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220 entries, 0 to 2219
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   행정구역(시군구)별  2220 non-null   object
 1   시점          2220 non-null   object
 2   총인구수 (명)    2220 non-null   object
 3   남자인구수 (명)   2220 non-null   object
 4   여자인구수 (명)   2220 non-null   object
dtypes: object(5)
memory usage: 86.8+ KB


In [29]:
data_mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2406 entries, 0 to 2405
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   행정구역(시군구)별         2406 non-null   object
 1   시점                 2406 non-null   object
 2   총전입 (명)            2406 non-null   int64 
 3   총전출 (명)            2406 non-null   int64 
 4   순이동 (명)            2406 non-null   int64 
 5   시도내이동-시군구내 (명)     2406 non-null   int64 
 6   시도내이동-시군구간 전입 (명)  2406 non-null   int64 
 7   시도내이동-시군구간 전출 (명)  2406 non-null   int64 
 8   시도간전입 (명)          2406 non-null   int64 
 9   시도간전출 (명)          2406 non-null   int64 
dtypes: int64(8), object(2)
memory usage: 188.1+ KB


In [30]:
data_pop

,행정구역(시군구)별,시점,총인구수 (명),남자인구수 (명),여자인구수 (명)
0,전국,2011. 01,50534731,25318961,25215770
1,서울특별시,2011. 01,10312835,5110837,5201998
2,부산광역시,2011. 01,3566560,1772383,1794177
3,대구광역시,2011. 01,2512071,1255238,1256833
4,인천광역시,2011. 01,2761122,1391674,1369448
...,...,...,...,...,...
2215,전라북도,2021. 04,1796331,893415,902916
2216,전라남도,2021. 04,1844148,927667,916481
2217,경상북도,2021. 04,2635896,1328240,1307656
2218,경상남도,2021. 04,3329623,1676550,1653073


In [31]:
data_mv

,행정구역(시군구)별,시점,총전입 (명),총전출 (명),순이동 (명),시도내이동-시군구내 (명),시도내이동-시군구간 전입 (명),시도내이동-시군구간 전출 (명),시도간전입 (명),시도간전출 (명)
0,전국,2010. 01,631545,631545,0,221718,190804,190804,219023,219023
1,전국,2010. 02,783081,783081,0,265412,240035,240035,277634,277634
2,전국,2010. 03,882541,882541,0,324720,262284,262284,295537,295537
3,전국,2010. 04,709932,709932,0,276612,209063,209063,224257,224257
4,전국,2010. 05,663632,663632,0,259609,197069,197069,206954,206954
...,...,...,...,...,...,...,...,...,...,...
2401,제주특별자치도,2020. 11,7135,6737,398,3789,681,681,2665,2267
2402,제주특별자치도,2020. 12,9344,8692,652,5071,916,916,3357,2705
2403,제주특별자치도,2021. 01,9713,9770,-57,5517,999,999,3197,3254
2404,제주특별자치도,2021. 02,10561,10059,502,5392,973,973,4196,3694


In [32]:
# 시도 이름 통일
# data_pop['행정구역(시군구)별'].unique()

match_dict = {'전국': '전국', '서울특별시': '서울', '부산광역시': '부산', '대구광역시': '대구', 
              '인천광역시': '인천', '광주광역시': '광주', '대전광역시': '대전', '울산광역시': '울산', 
              '경기도': '경기', '강원도': '강원', '충청북도': '충북', '충청남도': '충남', 
              '전라북도': '전북', '전라남도': '전남', '경상북도': '경북', '경상남도': '경남', 
              '제주특별자치도': '제주', '세종특별자치시': '세종'}

data_pop['행정구역(시군구)별'] = data_pop['행정구역(시군구)별'].map(match_dict)
data_mv['행정구역(시군구)별'] = data_mv['행정구역(시군구)별'].map(match_dict)
print(data_pop['행정구역(시군구)별'].unique())
print(data_mv['행정구역(시군구)별'].unique())

['전국' '서울' '부산' '대구' '인천' '광주' '대전' '울산' '경기' '강원' '충북' '충남' '전북' '전남'
 '경북' '경남' '제주' '세종']
['전국' '서울' '부산' '대구' '인천' '광주' '대전' '울산' '세종' '경기' '강원' '충북' '충남' '전북'
 '전남' '경북' '경남' '제주']


In [33]:
# 컬럼명 변경

data_pop.rename(columns={'시점': '날짜', '행정구역(시군구)별': '지역'}, inplace=True)
data_mv.rename(columns={'시점': '날짜', '행정구역(시군구)별': '지역'}, inplace=True)

In [34]:
# 파일 추출
# Tableau Dashboard로 시각화 진행

data_pop.to_csv('인구수.csv', index=False, encoding='cp949')
data_mv.to_csv('인구이동.csv', index=False, encoding='cp949')